In [1]:
import pandas as pd

In [2]:
# final file name
file_name = '11-30 to 12-6 drop.xlsx'

In [3]:
col_names = ['Agent','Manager','TL','Building','Apt','BB','Deal Date','Lease Start Date','Rent','Comm']

# read old table
old_table = pd.read_excel('11-30 to 12-6 old.xlsx',parse_dates= True,header = None, names = col_names)
old_table['Apt'] = old_table['Apt'].astype(str).str.upper()

# read new table 
new_table = pd.read_excel('11-30 to 12-6 new.xlsx',parse_dates= True).drop('Unnamed: 0',axis = 1)
new_table['Apt'] = new_table['Apt'].astype(str).str.upper()

In [4]:
# create a new column for both tables, this column works as a unique id for same units

new_table['Building + Apt'] = new_table['Building']+' '+new_table['Apt'].astype(str)
old_table['Building + Apt'] = old_table['Building']+' '+old_table['Apt'].astype(str)

# merge old table and new table
old_new = pd.merge(old_table,new_table,on = 'Building + Apt', how = 'outer')

In [5]:
def diff_agent(old,new):
    '''Find the changes of agents
    
    As mentioned in CRM_Clean_Reports, same deal(units) may be done by different agents, and the agent may drop
    the deal and new agent take the spot, this function is to find the changes of agents
    
    Args:
        old: agent names of old table
        new: agent names of new table
    
    Returns:
        result: changes seperated by '/'
        
        '''
    names = []
    new_names = new.split('/')
    old_names = old.split('/')
    for i in old_names:
        if i not in new_names:
            names.append(i)
    result = '/'.join(names)
    return result


In [6]:
# There will be 3 kinds of conditions will be consider as changes:
# 1) All new deals, because agents input data into CRM late. (old NAN)
# 2) Deals disappeared, because agents deleted deals. (new NAN)
# 3) Modified deals info, comm changed or agent changed
for i,k in old_new.iterrows():
    if pd.isna(k['Agent']):
        old_new.loc[i,'Agent-Diff'] = k['Agent_Combined']
        old_new.loc[i,'Comm-Diff'] = k['Comm_y']
    elif pd.isna(k['Agent_Combined']):
        old_new.loc[i,'Agent-Diff'] = k['Agent']
        old_new.loc[i,'Comm-Diff'] = -k['Comm_x']
    else:
        diff_comm = k['Comm_y'] - k['Comm_x']
        if not(-3 <= diff_comm <= 3):
            old_new.loc[i,'Agent-Diff'] = diff_agent(old_new.loc[i,'Agent'],old_new.loc[i,'Agent_Combined'])
            old_new.loc[i,'Comm-Diff'] =diff_comm

old_new[['Agent-Diff','Comm-Diff']] = old_new[['Agent-Diff','Comm-Diff']].fillna('')

In [7]:
# map the changes into old table
for q,p in old_new.iterrows():
    if not pd.isna(p['Agent']):
        old_table.loc[old_table[old_table['Building + Apt'] == p['Building + Apt']].index, 'agent_diff'] = p['Agent-Diff']
        old_table.loc[old_table[old_table['Building + Apt'] == p['Building + Apt']].index, 'comm_diff']  = p['Comm-Diff']

In [8]:
# drop the 'unique id' column
old_table = old_table.drop('Building + Apt' , axis = 1)

In [9]:
# find out the deals that agents input into CRM late
deals_added = old_new[pd.isna(old_new['Agent'])][['Agent_Combined', 'Manager_Combined', 'TL_Combined', 'Building_y',
       'Apt_y', 'BalAndBoth', 'DealDate', 'LeaseStartDate', 'Rent_y', 'Comm_y']]
        

In [10]:
writer = pd.ExcelWriter(file_name, datetime_format='MM-DD-YYYY')

old_table.to_excel(writer, sheet_name = 'Main')

deals_added.to_excel(writer, sheet_name = 'added')

writer.close()